In [2]:
import numpy as np
import math
from fractions import Fraction
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.circuit.library import QFT

In [3]:
# Euclideans algo to find greatest common divider
def gcd(a,b):
    while b:
        a,b=b,a%b
    return a

In [4]:

def power(a,d,n):
    res=1
    a=a%n
    while d>0:
        if d%2 == 1:
            res=(res*a)%n
        a=(a*a)%n
        d//=2
    return res

In [9]:
# Quantum phase estimation algorithm
def qpe(a,N,qubits_num):
    qpe_circuit=QuantumCircuit(qubits_num+int(np.ceil(np.log2(N))),qubits_num)
    count_qubits=qpe_circuit.qubits[:qubits_num]
    target_qubits=qpe_circuit.qubits[qubits_num:]
    
    # Initialize target qubits to |1>
    qpe_circuit.x(target_qubits[0])
    
    # apply hardmard gate
    qpe_circuit.h(count_qubits)
    
    for i in range(qubits_num):
        controlled_u=QuantumCircuit(int(np.ceil(np.log2(N))),name=f"U_a^{2**i}")
        for _ in range(2**i):
            
            def c_amod15(a,power):
                if a not in [2,7,8,11,13]:
                    raise ValueError("A must be in [2,7,8,11,13]")
                u = QuantumCircuit(4)
                for iteration in range(power):
                    if a in [2,13]:
                        u.swap(0,1)
                        u.swap(1,2)
                        u.swap(2,3)
                    if a in [7,8]:
                        u.swap(3,2)
                        u.swap(2,1)
                        u.swap(1,0)
                    if a==11:
                        u.swap(1,3)
                        u.swap(0,2)
                    if a==7:
                        for q in range(4):
                            u.z(q)
                u=u.control(1)
                return u
            num_target_bits=int(np.ceil(np.log2(N)))
            if N==15:
                controlled_u=c_amod15(a,1)
                qpe_circuit.append(controlled_u,[count_qubits[i]]+target_qubits[:4])
            else :
                pass
    iqft=QFT(qubits_num,inverse=True)
    qpe_circuit.compose(iqft,count_qubits,inplace=True)
    qpe_circuit.measure(count_qubits,range(qubits_num))
    return qpe_circuit

In [17]:
def count(circuit):
    simulator=AerSimulator()
    compiled_circuit=transpile(circuit,simulator)
    job=simulator.run(compiled_circuit,shots=1024)
    result=job.result()
    counts=result.get_counts()
    return counts

In [21]:
# Shors Algorithm
def shors(N,qubits_num=8):
    if N % 2==0:
        return 2
    
    a=np.random.randint(2,N)
    g=gcd(a,N)
    if g>1:
        return g
    try_limit=10
    
    for _ in range(try_limit):
        qpe_circuit=qpe(a,N,qubits_num)
        counts=count(qpe_circuit)
        if not counts:
            continue
        most_freq_outcome=max(counts,key=counts.get)
        phase=int(most_freq_outcome,2)/(2**qubits_num)
        fraction=Fraction(phase).limit_denominator(N)
        r=fraction.denominator
        
        if r>0 and power(a,r,N)==1:
            if r%2==0:
                x=power(a,r//2,N)
                factor1=gcd(x-1,N)
                factor2=gcd(x+1,N)
                
                if 1< factor1<N and 1< factor2<N and factor1*factor2==N:
                    return factor1,factor2
                elif 1<factor2<N:
                    return factor1
                elif 1<factor2<N:
                    return factor2
        return None

In [63]:
N=15
factors=shors(N,qubits_num=7)
print("Factor : ",factors)

Factor :  (3, 5)
